In [1]:
import json
from elasticsearch import Elasticsearch, helpers
import datetime
from glob import glob
es = Elasticsearch(['http://34.95.230.124:9200/'])

def parse_date(tweet):
    tweet['created_at'] = datetime.datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    tweet['user']['created_at'] = datetime.datetime.strptime(tweet['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    if ('retweeted_status' in tweet):
        tweet['retweeted_status']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
        tweet['retweeted_status']['user']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    return tweet

In [17]:
forest500settings = {
    "settings":{
        "index":{
            "mapping":{
                "total_fields":{
                    "limit":"2000"
                }
            }
        }
    }
}
es.indices.create('forest500',body=forest500settings)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'forest500'}

In [24]:
companies_path = glob('/Users/luiz/OneDrive/forest500/output-company/*.json')
#companies_path = glob('../data/company/*.json')
for company_path in companies_path:
    with open(company_path) as f:
        tweets = json.load(f)
    
    print(company_path)

    for tweet in tweets:
        tweet = parse_date(tweet)
        tweet['user']['type'] = 'company'
        res = es.index(index="forest500", id=tweet['id'], body=tweet)

es.indices.refresh(index="forest500")

/Users/luiz/OneDrive/forest500/output-company/mondigroup.json
/Users/luiz/OneDrive/forest500/output-company/adidasfootball.json
/Users/luiz/OneDrive/forest500/output-company/frieslndcampina.json
/Users/luiz/OneDrive/forest500/output-company/Runkeeper.json
/Users/luiz/OneDrive/forest500/output-company/invivogroup.json
/Users/luiz/OneDrive/forest500/output-company/mrvoficial.json
/Users/luiz/OneDrive/forest500/output-company/batashoes.json
/Users/luiz/OneDrive/forest500/output-company/naturaandco.json
/Users/luiz/OneDrive/forest500/output-company/dsm.json
/Users/luiz/OneDrive/forest500/output-company/vfcorp.json
/Users/luiz/OneDrive/forest500/output-company/keringgroup.json
/Users/luiz/OneDrive/forest500/output-company/suguna_foods.json
/Users/luiz/OneDrive/forest500/output-company/total.json
/Users/luiz/OneDrive/forest500/output-company/CP_News.json
/Users/luiz/OneDrive/forest500/output-company/ClariantCDP.json
/Users/luiz/OneDrive/forest500/output-company/dunkinbrands.json
/Users/luiz/

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [25]:
companies_path = glob('/Users/luiz/OneDrive/forest500/output/*.json')
#companies_path = glob('../data/if/*.json')
for company_path in companies_path:
    with open(company_path) as f:
        tweets = json.load(f)
    
    print(company_path)

    for tweet in tweets:
        tweet = parse_date(tweet)
        tweet['user']['type'] = 'financial'
        res = es.index(index="forest500", id=tweet['id'], body=tweet)

es.indices.refresh(index="forest500")

/Users/luiz/OneDrive/forest500/output/MizuhoAmericas.json
/Users/luiz/OneDrive/forest500/output/santander_br.json
/Users/luiz/OneDrive/forest500/output/wellsfargo.json
/Users/luiz/OneDrive/forest500/output/ScotiabankMX.json
/Users/luiz/OneDrive/forest500/output/Manulife.json
/Users/luiz/OneDrive/forest500/output/landg_group.json
/Users/luiz/OneDrive/forest500/output/deutschebank.json
/Users/luiz/OneDrive/forest500/output/lazard.json
/Users/luiz/OneDrive/forest500/output/SchrodersPR.json
/Users/luiz/OneDrive/forest500/output/StanChartHelp.json
/Users/luiz/OneDrive/forest500/output/china_bank.json
/Users/luiz/OneDrive/forest500/output/GroupeBPCE.json
/Users/luiz/OneDrive/forest500/output/InvescoUS.json
/Users/luiz/OneDrive/forest500/output/bancosantander.json
/Users/luiz/OneDrive/forest500/output/itau.json
/Users/luiz/OneDrive/forest500/output/AXANext.json
/Users/luiz/OneDrive/forest500/output/AmericanEquity.json
/Users/luiz/OneDrive/forest500/output/invescoindia.json
/Users/luiz/OneDriv

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [ ]:
query = {
  "query": {
    "query_string": {
      "query": "SantanderAM_UK",
      "default_field": "user.screen_name"
    }
  }
}

res = es.search(index="forest500", body=query)


In [2]:
import pandas as pd

companies = pd.read_excel('../input/forest500.xlsx',sheet_name='company')
financial = pd.read_excel('../input/forest500.xlsx',sheet_name='financial')

for index,row in companies.iterrows():
    if(pd.isna(row['Handles'])):
        continue
    handles = row['Handles'].split(',')
    for handle in handles:
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name.keyword": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }


        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                    
                        'official_name': row['Name'],
                        'type': row['Type'],
                        'subsidary': row['Subsidary'],
                        'brands': str(row['Brands']).split(','),
                        'sectors': str(row['Company sectors']).split(','),
                        'headquarters': row['Headquarters'],
                        'region': row['HQ region'],
                        'commodities_powerbroker': str(row['Commodities (powerbroker_']).split(','),
                        'segments_powerbroker': str(row['Segments (powerbroker)']).split(','),
                        'commodities_other': str(row['Commodities (Other)']).split(','),
                        'segments_other': str(row['Segments (Other)']).split(','),
                        'signatories': str(row['Sign']).split(','),
                }
            }
            try:
                es.update(index='forest500', id=tweet['_id'], body=update_query)
            except:
                print(update_query)


3M


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


agdagro
adidas
adidasfootball
adidasoriginals
adidasUK
adidasrunning
adidasUS
adidasHoops
adidasGolf
adidasalerts
adidassoccer
Adient
AdityaBirlaGrp
Advance_Digital
AEON_JAPAN
AFASCL
Agrifirm
AholdDelhaize
AldiUK
Allana_Sons
havaianaseurope
osklen
mizunousa 
amazon
AmazonUK
AMOREPACIFIC_US
Amul_Coop
Arauco_cl
ADMupdates
arcor
ArlaFoodsUK
AshleyHomeStore
ASICSamerica
Runkeeper
ASICSaustralia
 ASICSTennis
ASICSeurope
ASICS_ZA
ASICSUK
ASICSvolleyball
ASICS_India
ABFIngredients
adeo
BASFCorporation
BASFAgro
BASFAgProducts
BASF
BataCompanyNZ
beidahuanggroup
Bertelsmann_com
BESTGroupUK
BestGroupInt
BFG_Logistics
BioMarGroup
2SFGofficial
BRF_Brasil
bricapar
bright_food
bunge
clarksshoes
Calbee_JP
CampbellSoupCo
MichaelKors
 jimmychoo
 versace
cargill
CargillEMEA
Cargill_K12Food
CargillGrainUS
CargillAnimalUK
foodsecureworld
CarrefourSA
CarrefourGroup
CarrefourUAE
CarrefourUAE
CarrefourKe
Carrefour_qa
Groupe_Casino
CencosudArg
pr_CP
MengniuGroup
CSCECNEWS
clariant
ClariantOilSvcs
ClariantMB
Cl

In [3]:
import pandas as pd

financial = pd.read_excel('../input/forest500.xlsx',sheet_name='financial')

for index,row in financial.iterrows():
    if(pd.isna(row['Handles'])):
        continue
    handles = row['Handles'].split(',')
    for handle in handles:
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name.keyword": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }

        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                        'official_name': row['Name'],
                        'type': row['Type'],
                        'subsidary': str(row['FI subsidary']),
                        'FI type': str(row['FI type']).split(','),
                        'headquarters': str(row['FI Headquarters']),
                        'region': str(row['HQ Region']),
                        'commodities': str(row['Commodities assessed for']).split(','),
                }
            }
            try:
                es.update(index='forest500', id=tweet['_id'], body=update_query)
            except:
                print(update_query)

ABNAMRO
Aegon
AMGFunds
Allianz
Allstate
AmericanCentury
AmericanEquity
amfam
AIGinsurance
ameriprise
AmBankMY
AQRCapital
ANZ_AU
AXA
AXA_Partners
AXANext
axainsurance
AXAIM
AXAVentures
BaillieGifford
bbva
BancodoBrasil
BankofAmerica
china_bank
BNYMellon
Barclays
blackrock
BlackRock_UK
 BLKPublicPolicy
BlackRockCA
BMO
bndes
bnpparibas
GroupeBPCE
bradesco
calpers
capitalgroup
charlesschwab
citi
commerzbank
commbank
rabobankgroup
creditagricole
cic
creditsuisse
dbsbank
deutschebank
dimensional
eatonvance
epfoaindia
farmcredit
farmermacnews
fidelity
fifththird
fti_global
gicsingapore 
goldmansachs
GSBSociety
guggenheimptnrs
homeloansbyhdfc
hsbc
icicibank
eximbank_id
ing_news
intesasanpaolo
InvescoCanada
invescoindia
InvescoUS
invescoemea
itau
janushenderson
jefferies
jpmorgan
kbank_live
kkr_co
kotakbankltd
Krungthai_Care
lbbw
lazard
landg_group
fti_global 
libertymutual
licindiaforever
lincolnfingroup
lbgplc
macquarie
Manulife
MassMutualUS
MetLife
MUFGAmericas
MizuhoAmericas
MorganStanley
N